In [1]:
import argparse
import copy
import os
import os.path as osp
import pprint 
import sys
import time
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import yaml

os.environ["OPEN3D_ML_ROOT"] = "/home/threedee/repos/Open3D-ML"

import open3d.ml as _ml3d

Using external Open3D-ML in /home/threedee/repos/Open3D-ML


In [2]:
basedir = "/home/threedee/repos/Open3D-ML"
datesetbase = "/home/threedee/datasets"

randlanet_smartlab_cfg = basedir + "/ml3d/configs/randlanet_smartlab.yml"
randlanet_semantickitti_cfg = basedir +"/ml3d/configs/randlanet_semantickitti.yml"
randlanet_s3dis_cfg = basedir + "/ml3d/configs/randlanet_s3dis.yml"

ckpt_path = basedir + "logs/RandLANet_SmartLab_tf/checkpoint/ckpt-11"

kwargs = {
    "framework": "tf",
    "device": "cuda",
    "dataset_path": datesetbase + "/SmartLab",
    "split": "test",
    "ckpt_path": None,
    "cfg_file": randlanet_smartlab_cfg,
}

args = type("args", (object,), kwargs)()

pprint.pprint(kwargs)

{'cfg_file': '/home/threedee/repos/Open3D-ML/ml3d/configs/randlanet_smartlab.yml',
 'ckpt_path': None,
 'dataset_path': '/home/threedee/datasets/SmartLab',
 'device': 'cuda',
 'framework': 'tf',
 'split': 'train'}


In [3]:
if args.framework == "torch":
    import open3d.ml.torch as ml3d
else:
    import open3d.ml.tf as ml3d
    import tensorflow as tf

def merge_cfg_file(cfg, args, extra_dict):
    if args.device is not None:
        cfg.pipeline.device = args.device
        cfg.model.device = args.device
    if args.split is not None:
        cfg.pipeline.split = args.split
    if args.dataset_path is not None:
        cfg.dataset.dataset_path = args.dataset_path
    if args.ckpt_path is not None:
        cfg.model.ckpt_path = args.ckpt_path

    return cfg.dataset, cfg.pipeline, cfg.model

device = args.device
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        if device == "cpu":
            tf.config.set_visible_devices([], "GPU")
        elif device == "cuda":
            tf.config.set_visible_devices(gpus[0], "GPU")
        else:
            idx = device.split(":")[1]
            tf.config.set_visible_devices(gpus[int(idx)], "GPU")
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
cfg = _ml3d.utils.Config.load_from_file(args.cfg_file)

Pipeline = _ml3d.utils.get_module("pipeline", cfg.pipeline.name, args.framework)
Model = _ml3d.utils.get_module("model", cfg.model.name, args.framework)
Dataset = _ml3d.utils.get_module("dataset", cfg.dataset.name)

cfg_dataset, cfg_pipeline, cfg_model = merge_cfg_file(cfg, args, None)

dataset = Dataset(**cfg_dataset)
model = Model(**cfg_model)
pipeline = Pipeline(model, dataset, **cfg_pipeline)

In [5]:
# show the first 100 frames using the visualizer
# vis = ml3d.vis.Visualizer()
# vis.visualize_dataset(dataset, "training")  # , indices=range(100)

In [6]:
pipeline.cfg_tb = {
    "readme": "readme",
    "cmd_line": "cmd_line",
    "dataset": pprint.pformat(cfg_dataset, indent=2),
    "model": pprint.pformat(cfg_model, indent=2),
    "pipeline": pprint.pformat(cfg_pipeline, indent=2),
}
print(pipeline.cfg_tb)

if args.split == "test":
    pipeline.run_test()
else:
    pipeline.run_train()

,  2.92it/s]
INFO - 2021-03-21 03:14:05,343 - semantic_segmentation - loss train: 0.640  eval: 456.644
INFO - 2021-03-21 03:14:05,343 - semantic_segmentation - acc train: 0.866  eval: 0.358
INFO - 2021-03-21 03:14:05,344 - semantic_segmentation - iou train: 0.747  eval: 0.164
INFO - 2021-03-21 03:14:05,347 - semantic_segmentation - === EPOCH 59/100 ===
validation: 100%|██████████| 5/5 [00:01<00:00,  2.86it/s]
INFO - 2021-03-21 03:14:14,893 - semantic_segmentation - loss train: 0.565  eval: 347.561
INFO - 2021-03-21 03:14:14,893 - semantic_segmentation - acc train: 0.874  eval: 0.349
INFO - 2021-03-21 03:14:14,893 - semantic_segmentation - iou train: 0.753  eval: 0.155
INFO - 2021-03-21 03:14:14,896 - semantic_segmentation - === EPOCH 60/100 ===
validation: 100%|██████████| 5/5 [00:01<00:00,  2.93it/s]
INFO - 2021-03-21 03:14:24,419 - semantic_segmentation - loss train: 0.538  eval: 405.414
INFO - 2021-03-21 03:14:24,420 - semantic_segmentation - acc train: 0.896  eval: 0.362
INFO - 202

In [7]:
# pipeline.load_ckpt(ckpt_path=ckpt_path)

# pipeline.run_test()

In [8]:
# data = test_split.get_data(0)
# attr = test_split.get_attr(0)
# print(attr)

# results = pipeline.run_inference(data)

# pred = (results['predict_labels']).astype(np.int32)
# scores = results['predict_scores']


In [9]:
test_split = dataset.get_split("test")

vis_points = []

for idx in tqdm(range(len(test_split)), desc="test"):

    attr = test_split.get_attr(idx)
    data = test_split.get_data(idx)
    viz_data = test_split.get_data_viz(idx)

    print(attr)
    results = pipeline.run_inference(data)

    pred = (results['predict_labels']).astype(np.int32)

    label = viz_data['label']
    pts = data['point']

    vis_d = {
        "name": attr['name'],
        "points": pts,
        "labels": label,
        "pred": pred,
    }

    vis_points.append(vis_d)

INFO - 2021-03-21 03:20:47,952 - smartlab - Found 6 pointclouds for test
test:   0%|          | 0/6 [00:00<?, ?it/s]


AttributeError: 'SmartLabSplit' object has no attribute 'get_data_viz'

In [ ]:
v = ml3d.vis.Visualizer()
lut = ml3d.vis.LabelLUT()

white = [225, 225, 225]
yellow = [221, 183, 50]
lblue = [54, 193, 221]
pink = [197, 55, 207]
green = [62, 217, 57]
purple = [65, 22, 109]

colors = {
    'white': [x / 255.0 for x in white],
    'yello': [x / 255.0 for x in yellow],
    'lblue': [x / 255.0 for x in lblue],
    'pink': [x / 255.0 for x in pink],
    'green': [x / 255.0 for x in green],
    'purple': [x / 255.0 for x in purple],
}

print(colors)
label_names = dataset.get_label_to_names()
print(label_names)

for (c, cv), (l, lv) in zip(colors.items(), label_names.items()):
    lut.add_label(lv, l, cv)

v.set_lut("labels", lut)
v.set_lut("pred", lut)

v.visualize(vis_points)